# 📘 Clase 17: Domain Driven Design

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/heldigard/unaula-IF0100-POO-II/blob/main/notebooks/unidad-02/clase-17-domain-driven-design.ipynb)

## 🎯 Objetivos de Aprendizaje

Al finalizar esta clase, seras capaz de:
- Entender los conceptos basicos de DDD
- Identificar entidades y value objects
- Crear agregados y raices de agregado
- Aplicar el patron Repository
- Diseñar con lenguaje ubicuo

---

## 📚 Teoria: Domain Driven Design

### ¿Que es DDD?

**DDD** (Domain Driven Design) es un enfoque de desarrollo que:
- Centra el diseño en el dominio del negocio
- Usa un **lenguaje ubicuo** compartido entre desarrolladores y expertos del dominio
- Estructura el codigo en capas bien definidas

### Bloques de Construccion

| Concepto | Descripcion | Ejemplo en TaskFlow |
|----------|-------------|---------------------|
| **Entity** | Objeto con identidad unica | Usuario, Proyecto, Tarea |
| **Value Object** | Objeto sin identidad, inmutable | Email, Direccion, Dinero |
| **Aggregate** | Grupo de entidades/objetos con raiz | Proyecto + Tareas |
| **Repository** | Acceso a agregados persistidos | UsuarioRepository |
| **Service** | Logica de dominio que no pertenece a una entidad | ServicioAutenticacion |

---

## 🏛️ Capas de DDD

In [ ]:
# ============================================
# ESTRUCTURA DE CAPAS DDD
# ============================================

# Capa de Dominio (Domain Layer)
# - Entidades
# - Value Objects
# - Repositorios (interfaces)
# - Servicios de dominio
# - Eventos de dominio

# Capa de Aplicacion (Application Layer)
# - Casos de uso
# - Servicios de aplicacion
# - DTOs

# Capa de Infraestructura (Infrastructure Layer)
# - Implementaciones de repositorios
# - Conexiones a BD
# - Frameworks externos

# Capa de Presentacion (Presentation Layer)
# - API REST
# - Controllers
# - Views

print('✅ Estructura de capas DDD')

---

## 👤 Entidades

In [ ]:
# ============================================
# ENTIDADES (Tienen identidad)
# ============================================

from dataclasses import dataclass, field
from datetime import datetime
from typing import List
from abc import ABC, abstractmethod

class Entity(ABC):
    """Clase base para todas las entidades."""
    
    def __init__(self, id: int):
        self._id = id
    
    @property
    def id(self) -> int:
        return self._id
    
    def __eq__(self, other):
        if not isinstance(other, Entity):
            return False
        return self._id == other._id
    
    def __hash__(self):
        return hash(self._id)

class Usuario(Entity):
    """Entidad Usuario del dominio."""
    
    def __init__(self, id: int, username: str, email: str):
        super().__init__(id)
        self._username = username
        self._email = email
        self._activo = True
        self._fecha_registro = datetime.now()
    
    @property
    def username(self) -> str:
        return self._username
    
    @property
    def email(self) -> str:
        return self._email
    
    @property
    def activo(self) -> bool:
        return self._activo
    
    def desactivar(self):
        self._activo = False
    
    def activar(self):
        self._activo = True
    
    def __repr__(self):
        return f'Usuario({self.id}, {self._username})'

# Demostracion
usuario1 = Usuario(1, 'juan', 'juan@test.com')
usuario2 = Usuario(1, 'juan', 'juan@test.com')  # Mismo ID
usuario3 = Usuario(2, 'ana', 'ana@test.com')

print(f'usuario1 == usuario2: {usuario1 == usuario2}')  # True (mismo ID)
print(f'usuario1 == usuario3: {usuario1 == usuario3}')  # False

---

## 💎 Value Objects

In [ ]:
# ============================================
# VALUE OBJECTS (Sin identidad, inmutables)
# ============================================

@dataclass(frozen=True)
class Email:
    """Value Object para emails validos."""
    valor: str
    
    def __post_init__(self):
        if '@' not in self.valor:
            raise ValueError('Email invalido')
    
    def dominio(self) -> str:
        return self.valor.split('@')[1]

@dataclass(frozen=True)
class Dinero:
    """Value Object para cantidades monetarias."""
    cantidad: float
    moneda: str = 'USD'
    
    def __add__(self, other: 'Dinero') -> 'Dinero':
        if self.moneda != other.moneda:
            raise ValueError('No se pueden sumar monedas diferentes')
        return Dinero(self.cantidad + other.cantidad, self.moneda)
    
    def __repr__(self):
        return f'{self.moneda} {self.cantidad:.2f}'

# Demostracion
email = Email('usuario@example.com')
print(f'Email: {email.valor}')
print(f'Dominio: {email.dominio()}')

# Son inmutables
try:
    email.valor = 'otro@email.com'  # Error!
except AttributeError as e:
    print(f'Inmutable: {e}')

# Comparacion por valor
dinero1 = Dinero(100, 'USD')
dinero2 = Dinero(100, 'USD')
dinero3 = Dinero(50, 'USD')

print(f'\ndinero1 == dinero2: {dinero1 == dinero2}')  # True
print(f'dinero1 + dinero3: {dinero1 + dinero3}')  # USD 150.00

---

## 📦 Agregados

In [ ]:
# ============================================
# AGREGADOS (Raiz de agregado)
# ============================================

class Tarea(Entity):
    """Entidad hijo dentro del agregado Proyecto."""
    
    def __init__(self, id: int, titulo: str, descripcion: str = ''):
        super().__init__(id)
        self._titulo = titulo
        self._descripcion = descripcion
        self._estado = 'pendiente'
    
    def completar(self):
        self._estado = 'completada'
    
    def en_progreso(self):
        self._estado = 'en_progreso'
    
    @property
    def estado(self):
        return self._estado

class Proyecto(Entity):
    """
    Raiz de agregado.
    Mantiene consistencia de todas las tareas.
    """
    
    def __init__(self, id: int, nombre: str, creador_id: int):
        super().__init__(id)
        self._nombre = nombre
        self._creador_id = creador_id
        self._tareas: List[Tarea] = []
        self._activo = True
    
    def agregar_tarea(self, tarea: Tarea):
        """Solo a traves de la raiz se agregan tareas."""
        self._tareas.append(tarea)
    
    def obtener_tarea(self, tarea_id: int) -> Tarea:
        for tarea in self._tareas:
            if tarea.id == tarea_id:
                return tarea
        raise ValueError('Tarea no encontrada')
    
    def completar_tarea(self, tarea_id: int):
        tarea = self.obtener_tarea(tarea_id)
        tarea.completar()
    
    @property
    def progreso(self) -> float:
        if not self._tareas:
            return 0.0
        completadas = sum(1 for t in self._tareas if t.estado == 'completada')
        return completadas / len(self._tareas) * 100
    
    @property
    def tareas(self) -> List[Tarea]:
        return self._tareas.copy()

# Demostracion
proyecto = Proyecto(1, 'Mi Proyecto', 100)

# Agregar tareas solo a traves del agregado
proyecto.agregar_tarea(Tarea(1, 'Tarea 1'))
proyecto.agregar_tarea(Tarea(2, 'Tarea 2'))
proyecto.agregar_tarea(Tarea(3, 'Tarea 3'))

print(f'Total tareas: {len(proyecto.tareas)}')
print(f'Progreso: {proyecto.progreso:.0f}%')

# Completar tarea a traves del agregado
proyecto.completar_tarea(1)
proyecto.completar_tarea(2)

print(f'Progreso actualizado: {proyecto.progreso:.0f}%')

---

## 🗄️ Repositorios

In [ ]:
# ============================================
# REPOSITORIOS (Patron Repository)
# ============================================

from abc import ABC, abstractmethod
from typing import Optional, List

class RepositorioUsuario(ABC):
    """Interfaz del repositorio."""
    
    @abstractmethod
    def guardar(self, usuario: Usuario) -> None:
        pass
    
    @abstractmethod
    def buscar_por_id(self, id: int) -> Optional[Usuario]:
        pass
    
    @abstractmethod
    def buscar_por_email(self, email: str) -> Optional[Usuario]:
        pass
    
    @abstractmethod
    def listar_todos(self) -> List[Usuario]:
        pass

class RepositorioUsuarioEnMemoria(RepositorioUsuario):
    """Implementacion en memoria (para testing)."""
    
    def __init__(self):
        self._usuarios: dict = {}
    
    def guardar(self, usuario: Usuario) -> None:
        self._usuarios[usuario.id] = usuario
    
    def buscar_por_id(self, id: int) -> Optional[Usuario]:
        return self._usuarios.get(id)
    
    def buscar_por_email(self, email: str) -> Optional[Usuario]:
        for usuario in self._usuarios.values():
            if usuario.email == email:
                return usuario
        return None
    
    def listar_todos(self) -> List[Usuario]:
        return list(self._usuarios.values())

# Uso
repo = RepositorioUsuarioEnMemoria()

usuario = Usuario(1, 'juan', 'juan@test.com')
repo.guardar(usuario)

encontrado = repo.buscar_por_id(1)
print(f'Encontrado: {encontrado}')

por_email = repo.buscar_por_email('juan@test.com')
print(f'Por email: {por_email}')

---

## 🔗 Conexion con TaskFlow

TaskFlow esta construido con DDD:

```
src/taskflow/
├── domain/              # Capa de dominio
│   ├── models/          # Entidades
│   ├── value_objects/   # Value objects
│   └── repositories/    # Interfaces de repositorios
├── application/         # Capa de aplicacion
│   └── services/        # Casos de uso
├── infrastructure/      # Capa de infraestructura
│   └── repositories/    # Implementaciones
└── api/                 # Capa de presentacion
    └── routes/          # Endpoints
```

---

**¡Modela tu dominio, el codigo seguira! 🏛️**